In [ ]:
%pip install numpy
%pip install pandas
%pip install scikit-learn

import sys
import os

# Add the parent directory of 'src' to sys.path
sys.path.append(os.path.abspath('../src'))

# Importação de bibliotecas
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from neural_network.utils import one_hot_encode, normalize_data, plot_confusion_matrix, plot_learning_curve
from neural_network.losses import CategoricalCrossEntropyLoss
from neural_network.layers.dense_layer import DenseLayer
from neural_network.layers.dropout_layer import DropoutLayer
from neural_network.optimizer import GradientDescent

Traceback (most recent call last):
  File "c:\Users\acaua\.vscode\extensions\ms-python.python-2024.20.0-win32-x64\python_files\python_server.py", line 130, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 1
    %pip install numpy
    ^
SyntaxError: invalid syntax



In [ ]:
# Carregar o arquivo CSV
data = pd.read_csv('../data/multiclass/online_gaming_behavior.csv')

# Exibir primeiras linhas do dataset
print(data.head())

In [ ]:
# Selecionar colunas categóricas
categorical_columns = ['Gender', 'Location', 'GameGenre', 'GameDifficulty', 'EngagementLevel']

# Mapear cada categoria para um número
for column in categorical_columns:
    data[column] = data[column].astype('category').cat.codes

# Obter o número de classes para cada coluna categórica
num_classes = {column: data[column].nunique() for column in categorical_columns}

# Aplicar One-Hot Encoding
for column in categorical_columns:
    one_hot_encoded = one_hot_encode(data[column].values, num_classes[column])

    # Adicionar as colunas one-hot ao dataframe
    for i in range(num_classes[column]):
        data[f"{column}_{i}"] = one_hot_encoded[:, i]

    # Remover a coluna categórica original
    data = data.drop(column, axis=1)

# Exibir o dataset transformado
print(data.head())

In [ ]:
# Definir X (entrada) e y (saída)
X = data.drop(['PlayerID', 'EngagementLevel_0', 'EngagementLevel_1', 'EngagementLevel_2'], axis=1).values
y = one_hot_encode(data[['EngagementLevel_0', 'EngagementLevel_1', 'EngagementLevel_2']].values.argmax(axis=1), 3)

# Normalizar os dados de entrada
X = normalize_data(X)

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
class NeuralNetwork:
    def __init__(self, layers, loss, optimizer):
        self.layers = layers
        self.loss = loss
        self.optimizer = optimizer
        
    def forward(self, X):
        output = X
        for layer in self.layers:
            output = layer.forward(output)
        return output
    
    def backward(self, y_true, y_pred):
        grad = self.loss.backward(y_true, y_pred)
        for layer in reversed(self.layers):
            grad = layer.backward(grad)
    
    def update(self):
        for layer in self.layers:
            params = layer.get_parameters()
            if params:
                grads = layer.get_gradients()
                weights_updated, biases_updated = self.optimizer.update(
                    params['weights'], params['biases'],
                    grads['weights'], grads['biases']
                )
                layer.set_parameters({'weights': weights_updated, 'biases': biases_updated})

In [ ]:
# Definir a arquitetura da rede neural
input_size = X_train.shape[1]
output_size = y_train.shape[1]

layers = [
    DenseLayer(input_size=input_size, output_size=32, activation='relu'),
    DropoutLayer(rate=0.2),
    DenseLayer(input_size=32, output_size=24, activation='relu'),
    DropoutLayer(rate=0.2),
    DenseLayer(input_size=24, output_size=16, activation='relu'),
    DropoutLayer(rate=0.2),
    DenseLayer(input_size=16, output_size=8, activation='relu'),
    DropoutLayer(rate=0.2),
    DenseLayer(input_size=16, output_size=output_size, activation='softmax')
]

# Definir a função de perda e otimizador
loss = CategoricalCrossEntropyLoss()
optimizer = GradientDescent(learning_rate=0.01)

# Criar o modelo
model = NeuralNetwork(layers=layers, loss=loss, optimizer=optimizer)

In [ ]:
# Treinar a rede neural
epochs = 800
for epoch in range(epochs):
    y_pred = model.forward(X_train)
    loss_value = loss.calculate(y_train, y_pred)
    model.backward(y_train, y_pred)
    model.update()

    # Exibir métricas a cada 50 épocas
    if (epoch + 1) % 50 == 0:
        predictions = y_pred.argmax(axis=1)
        y_true = y_train.argmax(axis=1)
        acc = accuracy_score(y_true, predictions)
        print(f"Epoch {epoch + 1}, Loss: {loss_value:.4f}, Accuracy: {acc:.4f}")


In [ ]:
# Fazer previsões no conjunto de teste
y_pred_test = model.forward(X_test)
predictions_test = y_pred_test.argmax(axis=1)
y_true_test = y_test.argmax(axis=1)

# Calcular métricas de desempenho
print("\nClassification Report:")
print(classification_report(y_true_test, predictions_test))

# Acurácia
accuracy = accuracy_score(y_true_test, predictions_test)
print(f"Test Accuracy: {accuracy:.4f}")

In [ ]:
# Usar a função plot_confusion_matrix
print("Confusion Matrix:")
plot_confusion_matrix(y_test, predictions_test, labels=["Low", "Medium", "High"])

In [ ]:
# Usar a função plot_learning_curve
plot_learning_curve(model, X_train, y_train, accuracy)